In [1]:
# Basic Imports
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import pickle
import re
import collections
import tables as tb
import csv

In [16]:
#Paths
in_file = '/projects/nikhil/ADNI_prediction/input_datasets/CS/ADNIMERGE.csv' #Original File

#in_file = '/projects/nikhil/ADNI_prediction/input_datasets/ADNI_Merge_filter_Longitudinal.csv' #Selected Columns
in_data = pd.read_csv(in_file,header=0)
in_data.columns


Index([u'RID', u'PTID', u'VISCODE', u'SITE', u'COLPROT', u'ORIGPROT',
       u'EXAMDATE', u'DX_bl', u'AGE', u'PTGENDER', u'PTEDUCAT', u'PTETHCAT',
       u'PTRACCAT', u'PTMARRY', u'APOE4', u'FDG', u'PIB', u'AV45', u'CDRSB',
       u'ADAS11', u'ADAS13', u'MMSE', u'RAVLT_immediate', u'RAVLT_learning',
       u'RAVLT_forgetting', u'RAVLT_perc_forgetting', u'FAQ', u'MOCA',
       u'EcogPtMem', u'EcogPtLang', u'EcogPtVisspat', u'EcogPtPlan',
       u'EcogPtOrgan', u'EcogPtDivatt', u'EcogPtTotal', u'EcogSPMem',
       u'EcogSPLang', u'EcogSPVisspat', u'EcogSPPlan', u'EcogSPOrgan',
       u'EcogSPDivatt', u'EcogSPTotal', u'Ventricles', u'Hippocampus',
       u'WholeBrain', u'Entorhinal', u'Fusiform', u'MidTemp', u'ICV', u'DX',
       u'EXAMDATE_bl', u'CDRSB_bl', u'ADAS11_bl', u'ADAS13_bl', u'MMSE_bl',
       u'RAVLT_immediate_bl', u'RAVLT_learning_bl', u'RAVLT_forgetting_bl',
       u'RAVLT_perc_forgetting_bl', u'FAQ_bl', u'Ventricles_bl',
       u'Hippocampus_bl', u'WholeBrain_bl', u'Entorhi

In [22]:
col_list = ['PTID','AGE','PTGENDER','WholeBrain','Hippocampus']
cohort = 'ADNI1'
adni1_PTIDs = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'bl')][col_list]
ADNI1_age_file = '/projects/nikhil/ADNI_prediction/input_datasets/CS/{}_age.pkl'.format(cohort)
ADNI1_sex_file = '/projects/nikhil/ADNI_prediction/input_datasets/CS/{}_sex.pkl'.format(cohort)

cohort = 'ADNI2'
adni2_PTIDs = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'bl')][col_list]
common_ids = list(set(adni1_PTIDs)&set(adni2_PTIDs))
ADNI2_age_file = '/projects/nikhil/ADNI_prediction/input_datasets/CS/{}_age.pkl'.format(cohort)
ADNI2_sex_file = '/projects/nikhil/ADNI_prediction/input_datasets/CS/{}_sex.pkl'.format(cohort)

print len(adni1_PTIDs), len(adni2_PTIDs), len(common_ids)
#pickleIt(adni1_PTIDs, '/projects/nikhil/ADNI_prediction/input_datasets/CS/adni1_covar.pkl')
#pickleIt(adni2_PTIDs, '/projects/nikhil/ADNI_prediction/input_datasets/CS/adni2_covar.pkl')

adni1_PTIDs['PTGENDER'] = adni1_PTIDs['PTGENDER'].map({'Female': 1, 'Male': 0})
adni2_PTIDs['PTGENDER'] = adni2_PTIDs['PTGENDER'].map({'Female': 1, 'Male': 0})

adni1_age_dict = adni1_PTIDs.set_index('PTID')['AGE'].to_dict()
adni2_age_dict = adni2_PTIDs.set_index('PTID')['AGE'].to_dict()
adni1_sex_dict = adni1_PTIDs.set_index('PTID')['PTGENDER'].to_dict()
adni2_sex_dict = adni2_PTIDs.set_index('PTID')['PTGENDER'].to_dict()


print adni1_PTIDs



#save_dictionary(adni1_age_dict,ADNI1_age_file)
#save_dictionary(adni2_age_dict,ADNI2_age_file)
save_dictionary(adni1_sex_dict,ADNI1_sex_file)
save_dictionary(adni2_sex_dict,ADNI2_sex_file)

819 787 5
            PTID   AGE  PTGENDER  WholeBrain  Hippocampus
0     011_S_0002  74.3         0     1229740         8336
1     011_S_0003  81.3         0     1129830         5319
5     022_S_0004  67.5         0     1154980         6869
10    011_S_0005  73.7         0     1116630         7075
15    100_S_0006  80.4         1      927510         5348
20    022_S_0007  75.4         0      875798         6729
22    011_S_0008  84.5         1      948684         6080
24    011_S_0010  73.9         1     1033540         5485
28    022_S_0014  78.5         1      861749         6730
33    100_S_0015  80.8         0      942730         6732
36    011_S_0016  65.4         0      936539         7309
41    067_S_0019  73.1         1      912041         6647
45    011_S_0021  72.6         1      984357         8147
49    011_S_0022  63.2         0      886395         6410
53    011_S_0023  71.7         0     1040560         8310
60    067_S_0029  64.1         0     1222380         7359
64  

In [3]:
#Some Defs
def pickleIt(my_data,save_path):
    f = open(save_path, 'wb')
    pickle.dump(my_data, f)
    f.close()

def save_dictionary(_dict,save_path):
    f = open(save_path, 'wb')
    pickle.dump(_dict, f)
    f.close()
    
def save_list(_list, save_path):
    ofile  = open(save_path, "wb")
    writer = csv.writer(ofile,quoting=csv.QUOTE_NONE,delimiter='\n')
    writer.writerow(_list)
    #for item in _list:
    #    writer.writerow([item])
    ofile.close()
    
def getDemographics(master_table, cohort, timepoint, sub_list, feature_list):
    demographic_table = master_table[(in_data.ORIGPROT==cohort) & (in_data.COLPROT==cohort) & 
                                     (in_data.VISCODE==timepoint) & (in_data['PTID'].isin(sub_list))][feature_list]
    return demographic_table

In [23]:
# Generate demographics table for reporting purposes (papers!)
exp_name = 'Exp6'
cohort = 'ADNI1'
timepoint = 'bl'
feature_list = ['PTID','ADAS13','AGE','DX_bl','PTGENDER','PTEDUCAT','MMSE']

sub_list_path = '/projects/nikhil/ADNI_prediction/input_datasets/exp_data/CV_{}_{}_ADAS13.pkl'.format(exp_name,cohort)
pkl_file = open(sub_list_path, 'rb')
sub_list = pickle.load(pkl_file)['common_subs']
print len(sub_list)
demograph_table = getDemographics(in_data, cohort, timepoint, sub_list, feature_list)
print 'cohort: {}'.format(cohort)
#print demograph_table
print demograph_table['DX_bl'].value_counts()
print demograph_table['PTGENDER'].value_counts()
print 
print 'AGE: ({:4.1f},{:4.1f}), Edu: ({:4.1f},{:4.1f}), MMSE: ({:4.1f},{:4.1f}), ADAS-Cog: ({:4.1f},{:4.1f})'.format(
    float(demograph_table['AGE'].mean()),float(demograph_table['AGE'].std()),
    float(demograph_table['PTEDUCAT'].mean()),float(demograph_table['PTEDUCAT'].std()),
    float(demograph_table['MMSE'].mean()),float(demograph_table['MMSE'].std()),
    float(demograph_table['ADAS13'].mean()),float(demograph_table['ADAS13'].std()))
print

669
cohort: ADNI1
LMCI    326
CN      198
AD      145
Name: DX_bl, dtype: int64
Male      377
Female    292
Name: PTGENDER, dtype: int64

AGE: (75.0, 6.7), Edu: (15.5, 3.1), MMSE: (26.9, 2.6), ADAS-Cog: (18.0, 8.9)



In [ ]:
#Extract various cohorts

col_list = ['PTID','DX_bl','ADAS13','Hippocampus_bl','Hippocampus']
#col_list = ['PTID','DX_bl']
#col_list = ['PTID', 'DX_bl','DX']
#col_list = ['PTID']
cohort = 'ADNI2'
data_bl  = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'bl')][col_list]
data_m06 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm06')][col_list]
data_m12 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm12')][col_list]
data_m18 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm18')][col_list]
data_m24 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm24')][col_list]
data_m30 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm30')][col_list]
data_m36 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm36')][col_list]

In [ ]:
data_mx = data_m12
data_bl_cn = data_bl[(data_bl.DX_bl == 'CN')]
data_bl_ad = data_bl[(data_bl.DX_bl == 'AD')]
data_bl_mci = data_bl[(data_bl.DX_bl == 'LMCI')]
data_mx_nl = data_mx[(data_mx.DX == 'NL')]
data_mx_mci = data_mx[(data_mx.DX == 'MCI')]
data_mx_ad = data_mx[(data_mx.DX == 'Dementia')]

data_mx_conv = data_mx[((data_mx.DX == 'MCI to Dementia') | (data_mx.DX == 'Dementia')) & (data_mx.DX_bl == 'LMCI')]
data_mx_conv2 = data_mx[(data_mx.DX == 'Dementia') & (data_mx.DX_bl == 'LMCI')]
data_mx_nonconv = data_mx[(data_mx.DX == 'MCI') & (data_mx.DX_bl == 'LMCI')]
print 'bl_(cn,ad,mci):{} {} {}, mx_(cn,ad,mci):{} {} {}, mx_(conv, conv2, nonconv): {} {} {}'.format(
                len(data_bl_cn),len(data_bl_ad),len(data_bl_mci), 
                len(data_mx_nl),len(data_mx_ad),len(data_mx_mci), 
                len(data_mx_conv), len(data_mx_conv2), len(data_mx_nonconv))

MCI_CONV_PTID_DX_dict = data_mx_conv.set_index('PTID')['DX'].to_dict()
MCI_NONCONV_PTID_DX_dict = data_mx_nonconv.set_index('PTID')['DX'].to_dict()

#print MCI_CONV_PTID_DX_dict.keys()
#print MCI_NONCONV_PTID_DX_dict.keys()

MCI_Dx_conversion_IDs = {'MCI_conv':MCI_CONV_PTID_DX_dict.keys(), 'MCI_nonconv':MCI_NONCONV_PTID_DX_dict.keys()}

#print MCI_Dx_conversion_IDs
save_dict = True
if save_dict: #Save data subsets
    save_dict_path = '/projects/nikhil/jlf/revision_data/adni_vol/MCI_1year_Dx_conversion_IDs.pkl'
    save_dictionary(MCI_Dx_conversion_IDs,save_dict_path)

In [ ]:
#sanity check with the HC vol subject list
ADNI1_HC_file = '/projects/nikhil/ADNI_prediction/input_datasets/HC/subject_HC_vol_dictionary_ADNI1_left.pkl'
pkl_file = open(ADNI1_HC_file, 'rb')
ADNI1_HC_data = pickle.load(pkl_file)
print len(ADNI1_HC_data)
HC_subs = BL_PTID_DX_bl_dict.keys()
print HC_subs[:5]

In [ ]:
BL_dict = data_bl.set_index('PTID')['DX_bl'].to_dict()
print len(BL_dict)

CS_subs = BL_dict.keys()

#diff_subs = list(set(CS_subs) - set(HC_subs))

print CS_subs[:5]
#print len(diff_subs)

save_dict = True
if save_dict: #Save data subsets
    save_dict_path = '/projects/nikhil/ADNI_prediction/input_datasets/CS/ADNI2_BL_PTID_DX_bl_dict.pkl'
    save_dictionary(BL_dict,save_dict_path)
    
save_sub_list = False
if save_sub_list:
    sub_list = CS_subs
    save_ADNI2_sub_list_path = '/projects/nikhil/ADNI_prediction/input_datasets/CS/ADNIMERGE_ADNI2_BL_subject_list.txt'
    save_list(sub_list,save_ADNI2_sub_list_path)
    

In [ ]:
#Rename Columns
data_bl.columns=['PTID','DX_bl','bl_score']
data_m06.columns=['PTID','DX_bl','m06_score']
data_m12.columns=['PTID','DX_bl','m12_score']
data_m18.columns=['PTID','DX_bl','m18_score']
data_m24.columns=['PTID','DX_bl','m24_score']
data_m30.columns=['PTID','DX_bl','m30_score']
data_m36.columns=['PTID','DX_bl','m36_score']

In [ ]:
#Compute Joins / Unions / Intersections
test_merge = pd.merge(data_bl, data_m06, on='PTID',how='inner')
test_merge = pd.merge(test_merge, data_m12, on='PTID',how='inner')
test_merge = pd.merge(test_merge, data_m18, on='PTID',how='inner')
test_merge = pd.merge(test_merge, data_m24, on='PTID',how='inner')
test_merge['m06_diff'] = test_merge['m06_score'] - test_merge['bl_score']
test_merge['m12_diff'] = test_merge['m12_score'] - test_merge['bl_score']
test_merge['m18_diff'] = test_merge['m18_score'] - test_merge['bl_score']
test_merge['m24_diff'] = test_merge['m24_score'] - test_merge['bl_score']


In [ ]:
#Plot configs
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)
plt.style.use('ggplot')

In [ ]:
#Plots
plot_datasets = ['bl_score','m06_score','m12_score','m18_score','m24_score','m06_diff','m12_diff','m18_diff','m24_diff']
n_timepts = 5
delta_timepts = n_timepts-1 
y_mean = []
y_err = []
dx='All'
for plt_ds in plot_datasets:    
    if dx == 'All':
        y = np.array(test_merge[plt_ds])
    else:
        y = np.array(test_merge[test_merge.DX_bl==dx][plt_ds])
    
    y_mean.append(np.mean(y))
    y_err.append( v(y))
    
#Plot the stats
font_small = 8
font_med = 16
font_large = 24

plt.subplot(1,2,1)
x = np.arange(n_timepts)
y_mean_trunc = y_mean[:n_timepts]
y_err_trunc = y_err[:n_timepts]
plt.errorbar(x,y_mean_trunc, yerr=y_err_trunc, linewidth=2, label='bl_dx: {}'.format(dx))
plt.xlim([-1,5])
plt.xticks(x,plot_datasets[:n_timepts],fontsize=font_med)
plt.xlabel('Timepoint',fontsize=font_med)
plt.ylabel('ADAS13 Score',fontsize=font_med)            
plt.legend(fontsize=font_med,loc=2)
plt.title('Longitudinal ADAS_score',fontsize=font_med)

plt.subplot(1,2,2)
x = np.arange(1,delta_timepts+1,1)
y_mean_trunc = y_mean[n_timepts:]
y_err_trunc = y_err[n_timepts:]
plt.errorbar(x,y_mean_trunc, yerr=y_err_trunc, linewidth=2, label='bl_dx: {}'.format(dx))
plt.xlim([-1,5])
plt.xticks(x,plot_datasets[n_timepts:],fontsize=font_med)
plt.xlabel('Timepoint',fontsize=font_med)
plt.ylabel('ADAS13 Score',fontsize=font_med)            
plt.legend(fontsize=font_med,loc=2)
plt.title('Longitudinal Change in ADAS_score',fontsize=font_med)


In [ ]:
ad = data_bl.DX_bl[(data_bl.DX_bl == 'AD')]
cn = data_bl.DX_bl[(data_bl.DX_bl == 'CN')]
lmci = data_bl.DX_bl[(data_bl.DX_bl == 'EMCI')]
emci = data_bl.DX_bl[(data_bl.DX_bl == 'LMCI')]

print len(ad),len(cn),len(lmci),len(emci)


In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)
plt.subplot(2,1,1)
adni1_HC_bl.hist(bins=100)
plt.title('ADNI1 HC Vol (ADNIMERGE) distribution')
plt.subplot(2,1,2)
adni2_HC_bl.hist(bins=100)
plt.title('ADNI2 HC Vol (ADNIMERGE) distribution')

#box_fig = plt.gcf()
#box_fig.savefig('/projects/nikhil/ADNI_prediction/input_datasets/CS/adas_distribution.png', bbox_inches='tight')


In [ ]:
in_data.Hippocampus_bl